In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client, metrics, wait
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
import dask.bag as db
import numpy as np
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from distributed import progress
import os
from matplotlib import use
use('agg')
os.environ['PROJ_LIB'] = '/home/rjackson/anaconda3/envs/cmac_env2/share/proj/'
import pyart
import netCDF4
import tarfile
import tempfile
import shutil
from netCDF4 import num2date
import json
from time import strftime, sleep
import os
%matplotlib inline
import argparse
import datetime
import glob
import importlib
import subprocess
import time

from cmac import (cmac, get_cmac_values, quicklooks_ppi,
                  get_sounding_times, get_sounding_file_name)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/h5py/__init__.py:36: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/h5py/__init__.py:36: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [2]:
def run_cmac_and_plotting(
    radar_file_path, sounding_times, config, clutter_file, out_radar, image_directory, bad_directory, meta_append,
    overwrite, dd_lobes):
    """ For dask we need the radar plotting routines all in one subroutine. """
    cmac_config = get_cmac_values(config)

    try:
        radar = pyart.io.read(radar_file_path)
    except TypeError:
        if bad_directory is None:
            path = os.path.expanduser('~') + '/' + 'type_error_radars/'
        else:
            path = bad_directory
        print(radar_file_path + ' has encountered TypeError!')
        if not os.path.exists(path):
            os.makedirs(path)
            subprocess.call('chmod -R g+rw ' + path, shell=True)
        shutil.move(radar_file_path, path)
        return

    radar_start_date = netCDF4.num2date(radar.time['data'][0],
                                        radar.time['units'])
    year_str = "%04d" % radar_start_date.year
    month_str = "%02d" % radar_start_date.month
    day_str = "%02d" % radar_start_date.day
    hour_str = "%02d" % radar_start_date.hour
    minute_str = "%02d" % radar_start_date.minute
    second_str = "%02d" % radar_start_date.second

    save_name = cmac_config['save_name']
    if out_radar is None:
        the_path = (os.path.expanduser('~') + '/'+ year_str + month_str
                    + second_str + '/')
    else:
        the_path = (out_radar + '/' + year_str +  month_str
                    + day_str)
    file_name = (the_path + '/' + save_name + '.'
                 + year_str + month_str + day_str + '.' + hour_str
                 + minute_str + second_str + '.nc')

    # If overwrite is False, checks to see if the cmac_radar file
    # already exists. If so, CMAC 2.0 is not used on the original radar file.
    if overwrite is False and os.path.exists(file_name) is True:
        print(file_name + ' already exists.')
        return

    if not os.path.exists(the_path):
        os.makedirs(the_path)
        subprocess.call('chmod -R g+rw ' + the_path, shell=True)

    # Load clutter files.
    if clutter_file is not None:
        clutter_file_path = clutter_file
        if verbose:
            print('## Loading clutter file ' + clutter_file_path)
        clutter = pyart.io.read(clutter_file_path)
        if verbose:
            print('## Reading dictionary...')
        clutter_field_dict = clutter.fields['xsapr_clutter']
        if verbose:
            print('## Adding clutter field..')
        try:
            radar.add_field(
                'xsapr_clutter', clutter_field_dict, replace_existing=True)
            del clutter
        except ValueError:
            del clutter
            return
    # Retrieve closest sonde in time to the time of the radar file.
    closest_time = min(
        sounding_times, key=lambda d: abs(d - radar_start_date))
    sonde_name = cmac_config['sonde_name']
    sonde_file = get_sounding_file_name(
        sonde_path, sonde_name, closest_time)
    sonde = netCDF4.Dataset(sonde_file)

    # Running the cmac code to produce a cmac_radar object.
    try:
        cmac_radar = cmac(radar, sonde, config, flip_velocity=True,
                          meta_append=meta_append,
                          verbose=verbose)
    except ValueError:
        del radar
        sonde.close()
        return

    # Free up some memory.
    del radar
    sonde.close()

    # Produce the cmac_radar file from the cmac_radar object.
    pyart.io.write_cfradial(file_name, cmac_radar)
    print('## A CMAC radar object has been created at ' + file_name)

    # Providing the image_directory and checking if it already exists.
    img_directory = (image_directory + '/' + year_str + month_str
                     + day_str + '.' + hour_str + minute_str + second_str)
    if not os.path.exists(img_directory):
        os.makedirs(img_directory)
        subprocess.call('chmod -R g+rw ' + img_directory, shell=True)

    # Producing all the cmac_radar quicklooks.
    quicklooks_ppi(cmac_radar, config,
                   image_directory=img_directory,
                   dd_lobes=dd_lobes)

    # Delete the cmac_radar object and move on to the next radar file.
    del cmac_radar
    return

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
radar_path = '/lustre/or-scratch/cades-arm/rjackson/sgpxsaprsecI4.00/'
sonde_path = '/lustre/or-scratch/cades-arm/proj-shared/sgpsondewnpnC1.b1/'
config = 'xsapr_i4_ppi'

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
cmac_config = get_cmac_values(config)

x_compass = cmac_config['x_compass']
if os.path.isdir(radar_path):
    radar_files = glob.glob(radar_path + '/**/*' + x_compass
                            + '*', recursive=True)
elif os.path.isfile(radar_path):
    with open(radar_path) as f:
        radar_files = f.readlines()
    radar_files = [x.strip() for x in radar_files]
else:
    raise IOError('The specified radar path does not exist!')
sonde_name = cmac_config['sonde_name']
sounding_times = get_sounding_times(sonde_path, sonde_name)

# Get dates of radar files from the file name.
radar_times = []
for file_name in radar_files:
    where_x = file_name.find(x_compass)

    radar_times.append(
        datetime.datetime.strptime(file_name[where_x+3:where_x+15],
                                   '%y%m%d%H%M%S'))

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
radar_files

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180901143702.RAWUF7J',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180915213645.RAW0YM7',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180903191504.RAWUGS3',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180902094714.RAWUFSZ',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180902010100.RAWUFHJ',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180901025231.RAWUEW9',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180910081309.RAW0VPT',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180915031539.RAW0Y3M',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180915192218.RAW0YK2',
 '/lustre/or-scratch/cades-arm/proj-shared/cmac/sgpxsaprsecI4.00/201809/XSE180906035645.RAWUHNE',
 '/lustre/or-scratch

In [44]:
bad_directory = None
out_radar = '/lustre/or-hydra/cades-scratch/rjackson/sgpxsaprcmacsurI4.c1'
image_directory = '/lustre/or-hydra/cades-scratch/rjackson/cmac/sgpxsaprcmacsurI4.c1.png'
meta_append = 'config'
overwrite = True
clutter_file = '/lustre/or-hydra/cades-scratch/proj-shared/cmac/raw_sgpxsaprppiI4.00/clutter_files/clutter_201710.nc'
verbose = False
dd_lobes = True

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
#cluster = PBSCluster(name='dask-worker', memory='128GB', cores=36, processes=6, interface='ib0', project='arm',
#                    walltime='03:00:00', job_extra=['-W group_list=cades-arm'])
cluster1 = PBSCluster(processes=6, cores=36, walltime='3:00:00', queue='arm_high_mem', interface='ib0',
                      memory='128GB', project='arm', job_extra=['-W group_list=cades-arm'] )
cluster1.scale(6*10)         # Ask for ten workers
client1 = Client(cluster1)  # Connect this local process to remote workers

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46134 instead
  expected, actual


In [33]:
del cluster

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
client1

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Connection method: Cluster object,Cluster type: PBSCluster
Dashboard: http://10.23.216.81:8787/status,
Dashboard: http://10.23.216.81:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.23.216.81:33317,Workers: 0
Dashboard: http://10.23.216.81:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [45]:
def com():
    from matplotlib import use
    use('agg')
client1.run(com)

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'tcp://10.23.217.71:34038': None,
 'tcp://10.23.217.71:39049': None,
 'tcp://10.23.217.71:44361': None,
 'tcp://10.23.217.71:44835': None,
 'tcp://10.23.217.71:46386': None,
 'tcp://10.23.217.71:46689': None,
 'tcp://10.23.217.72:39466': None,
 'tcp://10.23.217.72:41655': None,
 'tcp://10.23.217.72:42159': None,
 'tcp://10.23.217.72:42449': None,
 'tcp://10.23.217.72:42613': None,
 'tcp://10.23.217.72:44644': None,
 'tcp://10.23.217.81:33799': None,
 'tcp://10.23.217.81:34899': None,
 'tcp://10.23.217.81:35358': None,
 'tcp://10.23.217.81:36358': None,
 'tcp://10.23.217.81:38549': None,
 'tcp://10.23.217.81:42892': None,
 'tcp://10.23.217.82:33915': None,
 'tcp://10.23.217.82:35348': None,
 'tcp://10.23.217.82:38425': None,
 'tcp://10.23.217.82:38496': None,
 'tcp://10.23.217.82:41714': None,
 'tcp://10.23.217.82:46456': None,
 'tcp://10.23.217.86:38404': None,
 'tcp://10.23.217.86:40386': None,
 'tcp://10.23.217.86:42420': None,
 'tcp://10.23.217.86:44053': None,
 'tcp://10.23.217.86

In [46]:
#the_bag = db.from_sequence(radar_files)
the_function = lambda x: run_cmac_and_plotting(
    x, sounding_times=sounding_times, config='xsapr_i4_ppi',
    clutter_file=clutter_file, out_radar=out_radar, image_directory=image_directory,
    bad_directory=bad_directory, meta_append=meta_append,
    overwrite=overwrite, dd_lobes=dd_lobes)
futures1 = client1.map(the_function, radar_files)

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
#start computation in the background
progress(futures1)

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'ipywidgets'

In [53]:
cluster1.stop_all_jobs()

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'PBSCluster' object has no attribute 'stop_all_jobs'

In [51]:
futures1

/home/rjackson/anaconda3/envs/cmac_env2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[<Future: error, key: lambda-251e761f135503df6a3c0850220d41c1>,
 <Future: error, key: lambda-e77922485450b69db92cd21e7dbed004>,
 <Future: error, key: lambda-f9e4520855e3678cb4be1d9013fc5879>,
 <Future: error, key: lambda-d3a3f80bd502f4e390db9e394544e7c6>,
 <Future: error, key: lambda-e391a66d96af00a5aecf3db9db8ccdbe>,
 <Future: error, key: lambda-13110e015d7a047b5795eba31645a99e>,
 <Future: error, key: lambda-32b77ad33454919948dbe1abe50fc09a>,
 <Future: error, key: lambda-ec1a5787750d37f8944a08b40026b2d6>,
 <Future: error, key: lambda-d90b49bdcd9dfe29830ac08ef9687856>,
 <Future: error, key: lambda-fc66d180a4aa80528cc2ceb1a038e35e>,
 <Future: error, key: lambda-bce9fe5a40510fcd44a148dc54a7c830>,
 <Future: error, key: lambda-b0402a91191591bde55144fed8bb05f0>,
 <Future: error, key: lambda-6234ba34c6e44f81e71702464ca2c90f>,
 <Future: error, key: lambda-1ecc00cabb77cd4a3922dac108176e11>,
 <Future: error, key: lambda-eaf50cbdb70db271672aeef696b8f5a0>,
 <Future: error, key: lambda-998f93717a7